In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive
from sklearn.preprocessing import FunctionTransformer
from copy import copy
import pickle
import warnings
warnings.filterwarnings('ignore')

In [12]:
df=pd.read_csv('../data/games.csv')
new_df=df.copy()

In [13]:
def data_prep(df):
    def encoder_result(x):
        if x == 'Home' or x == 'Win':
            return 1
        elif x == 'Draw':
            return 0
        elif x == 'Away' or x == 'Lose':
            return -1
        else:
            return x
    def encoder_teams(x):
        with open ('list_premiere.sav', 'rb') as fp:
            teams = pickle.load(fp)
        return teams.index(x)
    df.drop(columns=['link'],inplace=True)
    df['result']=df['result'].apply(lambda x: encoder_result(x))
    df['home_team']=df['home_team'].apply(lambda x: encoder_teams(x))
    df['away_team']=df['away_team'].apply(lambda x: encoder_teams(x))
    df['home_form_1']=df['home_form_1'].apply(lambda x: encoder_result(x))
    df['home_form_2']=df['home_form_2'].apply(lambda x: encoder_result(x))
    df['home_form_3']=df['home_form_3'].apply(lambda x: encoder_result(x))
    df['home_form_4']=df['home_form_4'].apply(lambda x: encoder_result(x))
    df['home_form_5']=df['home_form_5'].apply(lambda x: encoder_result(x))
    df['away_form_1']=df['away_form_1'].apply(lambda x: encoder_result(x))
    df['away_form_2']=df['away_form_2'].apply(lambda x: encoder_result(x))
    df['away_form_3']=df['away_form_3'].apply(lambda x: encoder_result(x))
    df['away_form_4']=df['away_form_4'].apply(lambda x: encoder_result(x))
    df['away_form_5']=df['away_form_5'].apply(lambda x: encoder_result(x))
    return df

In [14]:
new_df=data_prep(new_df)
y=new_df.result
X=new_df.drop(columns=['result'])

In [16]:
X.shape[0]

294

In [11]:
model=ExtraTreesClassifier(criterion='entropy', max_features=0.7000000000000001,
                     min_samples_leaf=17, min_samples_split=13,
                     random_state=42)

In [12]:
model.fit(X,y)
with open('model.pkl','wb') as f:
    pickle.dump(model,f)

In [19]:
df_proba=pd.DataFrame(model.predict_proba(X),columns=['Away','Draw','Home'])

In [20]:
df=pd.concat([df,df_proba],axis=1)

In [21]:
df_final=df[['round','home_team','away_team','result','Away','Draw','Home']]
df_final=df_final.sort_values(by=['round'])

,round,home_team,away_team,result,Away,Draw,Home
293,1,Manchester United,Chelsea,Home,0.330944,0.307336,0.361719
284,1,Liverpool,Norwich City,Home,0.123479,0.172204,0.704316
285,1,West Ham United,Manchester City,Away,0.450540,0.321514,0.227946
286,1,AFC Bournemouth,Sheffield United,Draw,0.322576,0.406787,0.270637
287,1,Burnley,Southampton,Home,0.323610,0.311490,0.364900
...,...,...,...,...,...,...,...
4,30,West Ham United,Wolverhampton Wanderers,Away,0.455201,0.281123,0.263677
3,30,Brighton & Hove Albion,Arsenal,Home,0.314712,0.293648,0.391640
2,30,Watford,Leicester City,Draw,0.489076,0.303955,0.206969
1,30,Tottenham Hotspur,Manchester United,Draw,0.362355,0.339623,0.298022


In [22]:
with open ('dict_odds.sav', 'rb') as fp:
            team_games = pickle.load(fp)
odds=pd.read_csv('https://www.football-data.co.uk/mmz4281/1920/E0.csv')
odds['HomeTeam']=odds['HomeTeam'].apply(lambda x: team_games[x])
odds['AwayTeam']=odds['AwayTeam'].apply(lambda x: team_games[x])
odds=odds[['HomeTeam','AwayTeam','B365H','B365D','B365A',]]

In [23]:
def search_odds_ch(row):
    mask=(odds['HomeTeam'] == row['home_team']) & (odds['AwayTeam']==row['away_team'])
    return odds.loc[mask,['B365H']].values[0][0]
def search_odds_dw(row):
    mask=(odds['HomeTeam'] == row['home_team']) & (odds['AwayTeam']==row['away_team'])
    return odds.loc[mask,['B365D']].values[0][0]
def search_odds_aw(row):
    mask=(odds['HomeTeam'] == row['home_team']) & (odds['AwayTeam']==row['away_team'])
    return odds.loc[mask,['B365A']].values[0][0]

In [24]:
df_final['odds_home']=df_final.apply(lambda row: search_odds_ch(row), axis=1)
df_final['odds_draw']=df_final.apply(lambda row: search_odds_dw(row), axis=1)
df_final['odds_away']=df_final.apply(lambda row: search_odds_aw(row), axis=1)

In [25]:
df_final['my_away']=df_final['Away']*df_final['odds_away']
df_final['my_home']=df_final['Home']*df_final['odds_home']
df_final['my_draw']=df_final['Draw']*df_final['odds_draw']

In [26]:
df_final=df_final[['round', 'home_team', 'away_team', 'result', 'Away','odds_away','my_away','Draw','odds_draw','my_draw','Home','odds_home','my_home']]
df_final.head(5)

,round,home_team,away_team,result,Away,odds_away,my_away,Draw,odds_draw,my_draw,Home,odds_home,my_home
293,1,Manchester United,Chelsea,Home,0.330944,3.50,1.158305,0.307336,3.3,1.014210,0.361719,2.10,0.759611
284,1,Liverpool,Norwich City,Home,0.123479,19.00,2.346109,0.172204,10.0,1.722042,0.704316,1.14,0.802921
285,1,West Ham United,Manchester City,Away,0.450540,1.22,0.549658,0.321514,6.5,2.089840,0.227946,12.00,2.735358
286,1,AFC Bournemouth,Sheffield United,Draw,0.322576,3.60,1.161275,0.406787,3.6,1.464432,0.270637,1.95,0.527742
287,1,Burnley,Southampton,Home,0.323610,2.75,0.889929,0.311490,3.2,0.996768,0.364900,2.62,0.956037


In [27]:
def busca_ganho(rodada,fator_conservador,fator_arrojado,percentual_conservador,valor_aposta):
    df_rodada=df_final.loc[df['round']==rodada,:]
    df_rodada.columns
    df_away=df_rodada[['home_team', 'away_team', 'result', 'Away', 'odds_away','my_away']]
    df_away['aposta']='Away'
    df_away.columns=['home_team', 'away_team', 'result', 'proba', 'odds','probaxodds','aposta']
    df_home=df_rodada[['home_team', 'away_team', 'result', 'Home', 'odds_home','my_home']]
    df_home['aposta']='Home'
    df_home.columns=['home_team', 'away_team', 'result', 'proba', 'odds','probaxodds','aposta']
    df_draw=df_rodada[['home_team', 'away_team', 'result', 'Draw', 'odds_draw','my_draw']]
    df_draw['aposta']='Draw'
    df_draw.columns=['home_team', 'away_team', 'result', 'proba', 'odds','probaxodds','aposta']
    df_1=pd.concat([df_away,df_draw,df_home])
    df_1=df_1.reset_index(drop=True)
    def set_aposta(x):
        if x>=fator_conservador:
            return 'conservador'
        elif x>=fator_arrojado:
            return 'arrojado'
        else:
            return np.nan 
    df_1['tipo']=df_1['proba'].apply(lambda x: set_aposta(x))
    df_1.dropna(inplace=True)
    sum_conservaodor=df_1.loc[df_1['tipo']=='conservador','probaxodds'].sum()
    sum_arrojado=df_1.loc[df_1['tipo']=='arrojado','probaxodds'].sum()
    def valor_apostado(row):
        if row['tipo'] == 'conservador':
            x= valor_aposta*percentual_conservador*(row['probaxodds']/sum_conservaodor)
        else:
            x= valor_aposta*(1-percentual_conservador)*(row['probaxodds']/sum_arrojado)
        return x
    df_1['valor']=df_1.apply(lambda row: valor_apostado(row), axis=1)
    def valor_ganho(row):
        if row['aposta'] == row['result']:
            return row['valor']*row['odds']
        else:
            return 0
    df_1['ganho']=df_1.apply(lambda row: valor_ganho(row), axis=1)
    total_ganho=df_1['ganho'].sum()
    return total_ganho-valor_aposta

In [ ]:
fatores_conservador=[round(item,2) for item in np.arange(0.4, 0.75, 0.01)]
fatores_arrojado=[round(item,2) for item in np.arange(0.2, 0.45, 0.01)]
percentuais_conservador=[round(item,2) for item in np.arange(0.5, 0.9, 0.01)]
valor_aposta = 10
resultados={}
for fatores_conservador in fatores_conservador:
    for fator_arrojado in fatores_arrojado:
        for percentual_conservador in percentuais_conservador:
            name=f'FC: {fatores_conservador}, FA: {fator_arrojado}, PC: {percentual_conservador}'
            valor_ganho=[busca_ganho(rodada,fator_conservador,fator_arrojado,percentual_conservador,valor_aposta) for rodada in range(1,31)]
            resultados[name]=sum(valor_ganho)

In [ ]:
resultados

In [ ]:
max(resultados, key=resultados.get)

In [ ]:
resultados['FC: 0.45, FA: 0.35, PC: 0.55']

In [28]:
fator_conservador=.45
fator_arrojado=.35
percentual_conservador=.55
valor_aposta = 10
resultados={}
valor_ganho=[busca_ganho(rodada,fator_conservador,fator_arrojado,percentual_conservador,valor_aposta) for rodada in range(1,31)]

In [33]:
valor_aposta=50
df_rodada=df_final.loc[df['round']==30,:]
df_rodada.columns
df_away=df_rodada[['home_team', 'away_team', 'result', 'Away', 'odds_away','my_away']]
df_away['aposta']='Away'
df_away.columns=['home_team', 'away_team', 'result', 'proba', 'odds','probaxodds','aposta']
df_home=df_rodada[['home_team', 'away_team', 'result', 'Home', 'odds_home','my_home']]
df_home['aposta']='Home'
df_home.columns=['home_team', 'away_team', 'result', 'proba', 'odds','probaxodds','aposta']
df_draw=df_rodada[['home_team', 'away_team', 'result', 'Draw', 'odds_draw','my_draw']]
df_draw['aposta']='Draw'
df_draw.columns=['home_team', 'away_team', 'result', 'proba', 'odds','probaxodds','aposta']
df_1=pd.concat([df_away,df_draw,df_home])
df_1=df_1.reset_index(drop=True)
def set_aposta(x):
    if x>=fator_conservador:
        return 'conservador'
    elif x>=fator_arrojado:
        return 'arrojado'
    else:
        return np.nan 
df_1['tipo']=df_1['proba'].apply(lambda x: set_aposta(x))
df_1.dropna(inplace=True)
sum_conservaodor=df_1.loc[df_1['tipo']=='conservador','probaxodds'].sum()
sum_arrojado=df_1.loc[df_1['tipo']=='arrojado','probaxodds'].sum()
def valor_apostado(row):
    if row['tipo'] == 'conservador':
        x= valor_aposta*percentual_conservador*(row['probaxodds']/sum_conservaodor)
    else:
        x= valor_aposta*(1-percentual_conservador)*(row['probaxodds']/sum_arrojado)
    return x
df_1['valor']=df_1.apply(lambda row: valor_apostado(row), axis=1)
def valor_ganho(row):
    if row['aposta'] == row['result']:
        return row['valor']*row['odds']
    else:
        return 0
#df_1['ganho']=df_1.apply(lambda row: valor_ganho(row), axis=1)
#total_ganho=df_1['ganho'].sum()
df_1[['home_team','away_team', 'valor']]

,home_team,away_team,valor
0,Everton,Liverpool,5.254767
1,Aston Villa,Chelsea,3.990351
4,West Ham United,Wolverhampton Wanderers,5.483197
6,Watford,Leicester City,5.750973
7,Tottenham Hotspur,Manchester United,4.029964
8,Norwich City,Southampton,7.020713
12,AFC Bournemouth,Crystal Palace,6.094883
20,Newcastle United,Sheffield United,6.677867
23,Brighton & Hove Albion,Arsenal,5.697286
